In [13]:
SEASON = '1983-84'

In [14]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df['PlayerScore'] = p_df['PlayerScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,Larry Bird,BOS,PF,117.080302,27,79,77,38.3,9.6,19.5,...,26.7,7.9,5.6,13.6,0.215,5.6,2.0,7.6,7.3,62.0
2,Magic Johnson,LAL,PG,87.509763,24,67,66,38.3,6.6,11.6,...,19.5,6.7,3.6,10.2,0.191,4.8,1.8,6.6,5.6,54.0
3,Adrian Dantley,UTA,SF,80.091496,28,79,79,37.8,10.2,18.2,...,28.2,13.0,1.6,14.6,0.235,5.8,-1.5,4.2,4.7,45.0
4,Julius Erving,PHI,SF,78.107074,33,77,77,34.8,8.8,17.2,...,27.2,5.6,4.5,10.1,0.180,4.3,1.6,5.9,5.3,52.0
5,Isiah Thomas,DET,PG,72.264742,22,82,82,36.7,8.2,17.7,...,25.4,5.7,3.4,9.1,0.144,4.0,0.7,4.8,5.1,49.0
6,Sidney Moncrief,MIL,SG,71.471338,26,79,79,38.9,7.1,14.2,...,22.2,8.4,4.2,12.7,0.198,3.7,0.4,4.1,4.7,50.0
7,Bernard King,NYK,SF,68.294698,27,77,76,34.6,10.3,18.1,...,28.7,8.6,3.5,12.1,0.218,4.6,-0.2,4.5,4.3,47.0
8,Mark Aguirre,DAL,SF,62.429042,24,79,79,36.7,11.7,22.3,...,33.4,7.0,2.0,9.0,0.149,5.0,-1.2,3.7,4.1,43.0
9,Robert Parish,BOS,C,60.039058,30,80,79,35.8,7.8,14.3,...,21.3,5.7,4.8,10.5,0.175,1.8,0.1,1.9,2.8,62.0
10,Kiki Vandeweghe,DEN,PF,58.282654,25,78,71,35.1,11.5,20.6,...,27.8,9.7,0.6,10.3,0.181,5.4,-1.8,3.6,3.9,38.0
